In [ ]:
## Extracting the data and saving it as a CSV

In [1]:
import sys
sys.path.append('../sentiment_analysis')

# This is another way of doing the above, not sure which is better
sys.path.insert(0, '../mining')

from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from vader_sentiment_analysis import SentimentAnalyser
import pandas as pd

import connector
import query

/home/alexander/anaconda/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
df = pd.read_csv("../data/SELECT_ALL_FROM_all_messages.csv", header = 0)

/home/sophie/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
len(df)

1354178

In [16]:
list(df)

['Unnamed: 0',
 'user_id',
 'contact_id',
 'relationship',
 'channel',
 'sent_at',
 'message']

In [8]:
df_meta = pd.read_csv("../data/SELECT_ALL_FROM_all_messages_metadata.csv", header = 0)

In [9]:
len(df_meta)

1070395

In [4]:
# Not able to read this in. 
#df = pd.read_csv("../data/all_messages_cleaned.csv", encoding='utf8', engine='python', usecols = ['user_id'], header = 0)

In [10]:
df = df.drop(['Unnamed: 0'],axis=1)

In [2]:
import psycopg2
from sshtunnel import SSHTunnelForwarder

port = 5432
remote_bind_address = ('localhost', 5432)
local_bind_address = ('localhost', 5432)
user_name = 'analytics'
dbname = 'dreikanter_production'
remote_host = 'dreikanter.production.devguru.co'
remote_port = 22
conn = None
server = None


params = {
            'database': 'dreikanter_production',
            'user': 'analytics',
            'host': 'localhost',
            'port': 5432
        }

server = SSHTunnelForwarder((remote_host, remote_port),ssh_username=user_name,
                                    local_bind_address=local_bind_address,
                                    remote_bind_address=remote_bind_address)
server.start()
print "We are in!"

#"dbname=dreikanter_production user=analytics host = localhost port = 5432"
conn = psycopg2.connect(**params)

We are in!


In [20]:
conn.close()
server.stop()

In [80]:

sql_query = """SELECT * FROM all_msgs_tf;"""

dbconnection = connector.ConnectDB()
conn = dbconnection.get_connection()
q = query.Query(conn, sql_query)
query_list = q.get_query_list()
query_df = q.get_query_dataframe()
#q.write_df_to_csv()

#query_df['body'][6:16]
#query_df.head(22)

2016-08-15 16:18:49,082 | ERROR   | Couldn't open tunnel localhost:5432 <> localhost:5432 might be in use or destination not reachable.


In [81]:
query_df.shape

(2859488, 7)

In [94]:
df_no_email = query_df[query_df['channel'] != 'gmail']

In [83]:
df_no_email.shape
#print query_df[query_df.message.str.contains("Land Rover")]

(2629679, 7)

In [117]:
df_no_email = df_no_email[df_no_email['message'] != '        ']

In [118]:
df_no_email.shape

(2517804, 7)

In [119]:
b = [len(str(x).split()) for x in df_no_email['message']]

In [125]:
df_no_email['word_count'] = b

In [129]:
#df_no_email[df_no_email['user_id'] == 12284]

In [ ]:
sentiment_analyser = SentimentAnalyser()
result = sentiment_analyser.run_vader(df_no_email, 'message')

In [33]:
df_no_gmail.to_csv('all_messages_no_gmail_tf_wc.csv', sep='\t', encoding='utf-8')